Things to know before -

This is Logistic Regression from scratch. This is done and implemented in a way that this can easily be extended to Neural Networks.

For now, we can assume this to be a single Neuron with one output.

Formulas -

Compact form -

z = w.T X + b -> a = sigmoid(z) -> Loss l(a,y) = - [y * log(a) + (1-y) * log(1-a)]

Our goal - Find optimal w* and b* that minimizes the Loss.

Disclamer- We can write the whole formula in one line,
loss(w,b) =  -[ y * log(sigmoid(w.T X + b)) + (1-y) - log(1-sigmoid(w.T X+ b))]
We can minimize this also, but it gets a bit messy.

We are going to use the compact form to keep everything nice and clean.

Our goal is to get dl/dz. We can achieve that by finding dl/da * da/dz (chain rule).

dl/da= (a-y)/ a(1-a)

da/dz= a * (1-a)

=> dl/dz = a-y

Now, to update w after each iteration, we need to find dl/dw.

dl/dw = dl/dz * dz/dw

dz/dw = X

=> dl/dw= (a-y)*X

similarly update for b- 

dl/db = dl/dz*dz/db

dz/db= 1 

dl/db = a-y



***********************

**final update for all w's, w= 1/n * learning_rate * (a-y) * X**

**final update for all b's, w= 1/n * learning_rate * (a-y)**

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.metrics as m
from sklearn.linear_model import LogisticRegression

Matplotlib is building the font cache; this may take a moment.


In [14]:
def generate_data():
  X,y=make_classification(n_samples=1000,n_features=10,random_state=42) # Generating random data
  X=np.concatenate((X,np.ones(X.shape[0]).reshape(X.shape[0],1) ), axis=1)
  X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.33, random_state=42)
  lr=0.001
  lamb=0.01
  #print(X.shape,y.shape)
  return X_train, X_test, y_train, y_test,lr,lamb

We have 1000 samples with each sample having 10 features.

In [15]:
def sigmoid(z):
  a=1 / ( 1 + np.exp(-(z) ) ) 
  return a

In [16]:
def forward_prop(w,X):
  return sigmoid( w.T.dot(X) )

In [17]:
def back_prop(X,a,y):
  dw = np.dot(X,(a-y))
  return dw

In [18]:
def initialize_params(X):
  w=np.zeros(X.shape[1])
  #b=np.ones(X.shape[0])
  return w

In [19]:
def loss(y, y_hat):
    loss = -np.mean(y*(np.log(y_hat)) - (1-y)*np.log(1-y_hat)) + lamb * w**2
    return loss

In [20]:
def predict(w,X):
  return sigmoid( w.T.dot(X) )

In [21]:
if __name__=='__main__':
  X_train, X_test, y_train, y_test,lr,lamb=generate_data()
  w=initialize_params(X_train)
  w=w.T
  X_train=X_train.T
  X_test=X_test.T
  y_train=y_train.T
  y_test=X_test.T  

  lo=[]
  accuracy=[]
  for i in range(0,1000):
    predictions=[]
    a=forward_prop(w,X_train)
    dw=back_prop(X_train,a,y_train)
    w[0:10]= ((w[0:10] - ( lr * (dw[0:10]) )   ) + lamb* w[0:10] )/X_train.shape[0]
    w[10:11]=((w[10:11] - ( lr * (dw[10:11]) )   ) )/X_train.shape[0]
    for i in a:
      predictions.append(1 if i>=0.5 else 0)
    lo.append(loss(y_train,a))
    accuracy.append(m.accuracy_score(y_train,predictions))

In [27]:
a=predict(w,X_test)
predictions_test=[]
for i in a:
  predictions_test.append(1 if i>=0.5 else 0)
print('acc on test set',m.accuracy_score(y_test,predictions_test))

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

In [28]:
clf=LogisticRegression()
clf.fit(X_train,y_train)
preds=clf.predict(X_test)
print('sklearn accuracy on test',m.accuracy_score(y_test,preds))

ValueError: Found input variables with inconsistent numbers of samples: [11, 670]

In [29]:
y_test

array([[ 0.74030596,  0.42548993,  0.6039124 , ..., -0.98264957,
        -1.3716737 ,  1.        ],
       [-1.49577819,  0.38135283, -1.2763343 , ...,  1.80201193,
        -0.63014283,  1.        ],
       [-0.14407863, -0.00928856,  0.02366179, ...,  0.65257238,
         1.8134077 ,  1.        ],
       ...,
       [-1.67945551,  0.47314545, -1.40250885, ...,  2.12312866,
        -0.23780704,  1.        ],
       [-0.50417344, -0.36861937, -0.45662085, ...,  0.52109091,
         1.11968778,  1.        ],
       [ 0.30714244,  0.74661136,  0.45012969, ...,  0.24438526,
         2.27582543,  1.        ]])